In [1]:
pip install tf-keras pandas torch transformers scikit-learn ipywidgets sentence_transformers tqdm


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: /opt/homebrew/Cellar/jupyterlab/4.2.5_1/libexec/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertModel
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.svm import SVC
from sentence_transformers import SentenceTransformer
from tqdm.notebook import tqdm


In [3]:
# Load the dataset
df = pd.read_csv('WELFakeDataset5000.csv')  # Replace with your CSV file path

# Drop rows where the label is NaN
df = df.dropna(subset=['label'])


# Extract relevant columns
text_data = df['text'].fillna('').tolist()
title_data = df['title'].fillna('').tolist()
labels = df['label'].tolist()  # 1 = real, 0 = fake
combined_data = [f"{title} {text}" for title, text in zip(title_data, text_data)]
# Load BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')
print(df['label'].value_counts())

/var/folders/5t/fxgd6xw57pv3hcsy91dsxnkm0000gn/T/ipykernel_58989/3924009589.py:2: DtypeWarning: Columns (50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('WELFakeDataset5000.csv')  # Replace with your CSV file path


label
1                                                                                                               2206
0                                                                                                               2001
 anyone who thinks the Feminists want ‚Äúequality‚Äù only has to look at this decision. If Hillary was a man       1
 and not one shred of gratitude nor remorse from those who took credit they were unable to pay. Oz Steamer         1
 if current demographic trends continue                                                                            1
                                                                                                                ... 
 and whatever else I might decide to indulge in                                                                    1
 you‚Äôll help the Repubs steal what‚Äôs left of the country. Michael                                              1
 the safety net is going to break.‚Äù Yes                 

In [4]:
# Define function to get BERT embeddings
def get_bert_embedding(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).squeeze().numpy()  # Pool to get a single embedding



In [5]:

model = SentenceTransformer('all-MiniLM-L6-v2')


# Get embeddings for each text
#embeddings = [get_bert_embedding(title) for title in title_data]

#embeddings = [get_bert_embedding(text) for text in tqdm(combined_data)]

text_temp, text_test, y_temp, y_test = train_test_split(combined_data, labels, test_size=0.2, random_state=42)
text_train, text_val, y_train, y_val = train_test_split(text_temp, y_temp, test_size=0.25, random_state=42)

embeddings_train = [model.encode(text) for text in tqdm(text_train, desc="Generating embeddings for training set")]
embeddings_val = [model.encode(text) for text in tqdm(text_val, desc="Generating embeddings for validation set")]
embeddings_test = [model.encode(text) for text in tqdm(text_test, desc="Generating embeddings for test set")]




Generating embeddings for training set:   0%|          | 0/2640 [00:00<?, ?it/s]

Generating embeddings for validation set:   0%|          | 0/880 [00:00<?, ?it/s]

Generating embeddings for test set:   0%|          | 0/880 [00:00<?, ?it/s]

In [6]:
from sklearn.metrics import confusion_matrix
import numpy as np

# Convert embeddings to numpy arrays for processing
X_train = np.array(embeddings_train)
X_val = np.array(embeddings_val)
X_test = np.array(embeddings_test)


# Scale embeddings
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

# Increase max_iter in LogisticRegression / Train a classifier
#clf = RandomForestClassifier(n_estimators=100, random_state=42)
#clf = LogisticRegression(max_iter=500)  # Increase iterations

# Train an ensemble model
ensemble = VotingClassifier(estimators=[
    ('lr', LogisticRegression(max_iter=500)),
    ('rf', RandomForestClassifier(n_estimators=100, random_state=42)),
    ('svc', SVC(kernel='rbf', probability=True))
], voting='soft')
ensemble.fit(X_train_scaled, y_train)

# Validate the model
y_val_pred = ensemble.predict(X_val_scaled)
val_accuracy = accuracy_score(y_val, y_val_pred)
print(f"Validation Accuracy: {val_accuracy * 100:.2f}%")

# Test the model
y_test_pred = ensemble.predict(X_test_scaled)
test_accuracy = accuracy_score(y_test, y_test_pred)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

# Confusion matrix
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_test_pred))

# Classification report
print("\nClassification Report:")
print(classification_report(y_test, y_test_pred))



Validation Accuracy: 81.82%
Test Accuracy: 83.86%

Confusion Matrix:
 [[  0   0   0 ...   1   0   0]
 [  0   0   0 ...   1   0   0]
 [  0   0   0 ...   0   0   0]
 ...
 [  0   0   0 ... 383   0   0]
 [  0   0   0 ...   1   0   0]
 [  0   0   0 ...   0   0   0]]

Classification Report:
                                                                                                                                                                                                                                                                                                                                                                  precision    recall  f1-score   support

                                                                                                                                                  Duncan explained ‚ÄúBaiame came from a place that we call the Morning Star within the Mirrabooka. Mira means stars and booka means river. That is the Milky Way that flows ac

/opt/homebrew/Cellar/jupyterlab/4.2.5_1/libexec/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/homebrew/Cellar/jupyterlab/4.2.5_1/libexec/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/homebrew/Cellar/jupyterlab/4.2.5_1/libexec/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average

In [8]:
# Drop any unnamed columns (columns with no header) if they exist
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

# Add predictions to original dataframe
#df['prediction'] = df['prediction'].apply(lambda x: 'real' if x == 1 else 'fake')

# Save results
df.to_csv('bert_welfcombined.csv', index=False)
print("Analysis complete. Results saved'.")


Analysis complete. Results saved'.
